In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [17]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
```

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
from datetime import datetime

In [8]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [13]:
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)]

In [15]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), PULocationID=244, total_amount=22.8)

In [14]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [16]:
rdd.filter(prepare_for_grouping).map(prepare_for_grouping).take(1)

[((datetime.datetime(2020, 1, 22, 13, 0), 244), (22.8, 1))]

In [18]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [22]:
rdd.filter(prepare_for_grouping).map(prepare_for_grouping).reduceByKey(calculate_revenue).take(10)

[((datetime.datetime(2020, 1, 18, 11, 0), 128), (33.51, 1)),
 ((datetime.datetime(2020, 1, 15, 21, 0), 42), (336.1400000000001, 23)),
 ((datetime.datetime(2020, 1, 19, 0, 0), 25), (142.75000000000003, 11)),
 ((datetime.datetime(2020, 1, 24, 19, 0), 166), (739.61, 48)),
 ((datetime.datetime(2020, 1, 17, 9, 0), 145), (175.59999999999997, 19)),
 ((datetime.datetime(2020, 1, 30, 20, 0), 129), (227.53999999999996, 11)),
 ((datetime.datetime(2020, 1, 2, 14, 0), 31), (23.16, 1)),
 ((datetime.datetime(2020, 1, 8, 9, 0), 171), (146.44, 4)),
 ((datetime.datetime(2020, 1, 7, 12, 0), 242), (306.8, 15)),
 ((datetime.datetime(2020, 1, 3, 18, 0), 74), (1733.4099999999992, 100))]

In [23]:
from collections import namedtuple

In [24]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [25]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [26]:
rdd.filter(prepare_for_grouping).map(prepare_for_grouping).reduceByKey(calculate_revenue).map(unwrap).take(10)

[RevenueRow(hour=datetime.datetime(2020, 1, 18, 11, 0), zone=128, revenue=33.51, count=1),
 RevenueRow(hour=datetime.datetime(2020, 1, 15, 21, 0), zone=42, revenue=336.1400000000001, count=23),
 RevenueRow(hour=datetime.datetime(2020, 1, 19, 0, 0), zone=25, revenue=142.75000000000003, count=11),
 RevenueRow(hour=datetime.datetime(2020, 1, 24, 19, 0), zone=166, revenue=739.61, count=48),
 RevenueRow(hour=datetime.datetime(2020, 1, 17, 9, 0), zone=145, revenue=175.59999999999997, count=19),
 RevenueRow(hour=datetime.datetime(2020, 1, 30, 20, 0), zone=129, revenue=227.53999999999996, count=11),
 RevenueRow(hour=datetime.datetime(2020, 1, 2, 14, 0), zone=31, revenue=23.16, count=1),
 RevenueRow(hour=datetime.datetime(2020, 1, 8, 9, 0), zone=171, revenue=146.44, count=4),
 RevenueRow(hour=datetime.datetime(2020, 1, 7, 12, 0), zone=242, revenue=306.8, count=15),
 RevenueRow(hour=datetime.datetime(2020, 1, 3, 18, 0), zone=74, revenue=1733.4099999999992, count=100)]

In [27]:
from pyspark.sql import types

In [28]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [29]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [30]:
df_result.show(10)

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-18 11:00:00| 128|             33.51|    1|
|2020-01-15 21:00:00|  42| 336.1400000000001|   23|
|2020-01-19 00:00:00|  25|142.75000000000003|   11|
|2020-01-24 19:00:00| 166|            739.61|   48|
|2020-01-17 09:00:00| 145|175.59999999999997|   19|
|2020-01-30 20:00:00| 129|227.53999999999996|   11|
|2020-01-02 14:00:00|  31|             23.16|    1|
|2020-01-08 09:00:00| 171|            146.44|    4|
|2020-01-07 12:00:00| 242|             306.8|   15|
|2020-01-03 18:00:00|  74|1733.4099999999992|  100|
+-------------------+----+------------------+-----+
only showing top 10 rows



In [31]:
df_result.write.parquet('tmp/green-revenue')

In [32]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [33]:
import pandas as pd

In [34]:
rows = duration_rdd.take(10)

In [35]:
df = pd.DataFrame(rows, columns=columns)

In [37]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [38]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [71]:
def inifinite_seq():
    i = 0
    while True:
        if i>10:
            break
        yield i
        i = i+1

In [74]:
seq = inifinite_seq()
print(next(seq))
print(list(seq))
print(seq)

0
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
<generator object inifinite_seq at 0x00000275C73318C0>


In [39]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [40]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [75]:
df_predicts.select('predicted_duration').show(10)

+------------------+
|predicted_duration|
+------------------+
|26.099999999999998|
|              4.35|
|              3.15|
|             13.55|
|10.649999999999999|
|             89.65|
|120.19999999999999|
|              7.25|
|54.699999999999996|
|              15.9|
+------------------+
only showing top 10 rows



In [52]:
def apply_model_in_batch(partition):
    cnt = 0
    for row in partition:
        cnt = cnt + 1
    return [cnt]

In [54]:
len(duration_rdd.mapPartitions(apply_model_in_batch).collect())
# the number of partitions for duration_rdd

16